# Imports

In [97]:
import os
import re
import time
import torch
import chess.pgn
import numpy as np
import torch.nn as nn
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from torchsummary import summary
from torch.utils.data import DataLoader
from torch.utils.data import IterableDataset

# Paths

In [98]:
base_dir = "/home/mystic/Programming/mystic-bot/notebooks"
pgn_file = f"{base_dir}/data/lichess_db_standard_rated_2014-07.pgn"
pt_file = f"{base_dir}/data/lichess_2014_07.pt"
model_dir = f"{base_dir}/models"
plot_dir = f"{base_dir}/plots"
os.makedirs(model_dir, exist_ok=True)
os.makedirs(plot_dir, exist_ok=True)

# Device

In [99]:
if torch.cuda.is_available():
    device_count = torch.cuda.device_count()
    print(f"✅ {device_count} CUDA device(s) available:")
    for i in range(device_count):
        print(f"  └─ [{i}] {torch.cuda.get_device_name(i)}")
    device = torch.device("cuda")
else:
    print("⚠️ CUDA not available, using CPU.")
    device = torch.device("cpu")

✅ 1 CUDA device(s) available:
  └─ [0] NVIDIA GeForce GTX 1650


# Pre-Process Data

In [101]:
EVAL_REGEX = re.compile(r"\[%eval (-?\d+(\.\d+)?)\]")
ENCODE_PIECES = {
    'P': 0, 'N': 1, 'B': 2, 'R': 3, 'Q': 4, 'K': 5,
    'p': 6, 'n': 7, 'b': 8, 'r': 9, 'q': 10, 'k': 11
}


def encode_board(fen):
    board_tensor = np.zeros((12, 8, 8), dtype=np.float32)
    board = chess.Board(fen)
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece:
            idx = ENCODE_PIECES[piece.symbol()]
            row, col = divmod(square, 8)
            board_tensor[idx, 7 - row, col] = 1
    return board_tensor


def preprocess_pgn(pgn_path, output_path, max_games=1_000_000):
    if os.path.exists(pt_file):
        print(f"\n✅ PT file already exists!")
    boards, evals = [], []
    with open(pgn_path, encoding="utf-8") as f:
        pbar = tqdm(total=max_games, desc="Preprocessing PGN")
        games_processed = 0

        while games_processed < max_games:
            game = chess.pgn.read_game(f)
            pbar.update(1)
            if game is None:
                break

            board = game.board()
            added_this_game = 0

            for node in game.mainline():
                if "eval" in node.comment:
                    match = EVAL_REGEX.search(node.comment)
                    if match:
                        try:
                            score = float(match.group(1))
                            score = max(min(score, 10), -10)
                            if board.turn == chess.BLACK:
                                score = -score

                            board_tensor = encode_board(board.fen())
                            boards.append(board_tensor)
                            evals.append(score)
                            added_this_game += 1
                        except ValueError:
                            continue

                if node.move:
                    board.push(node.move)

            if added_this_game > 0:
                games_processed += 1

        pbar.close()

    boards = np.stack(boards)
    evals = np.array(evals, dtype=np.float32).reshape(-1, 1)

    print(f"\n✅ Saved {len(boards)} positions from {games_processed} games.")

    # Save to disk
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    torch.save({"boards": torch.tensor(boards),
               "evals": torch.tensor(evals)}, output_path)

preprocess_pgn(pgn_file, pt_file, max_games=1_048_440)

Preprocessing PGN:   0%|          | 0/1000000 [00:00<?, ?it/s]

KeyboardInterrupt: 

# Dataset Encoder

In [ ]:
class EvalDataset(torch.utils.data.Dataset):
    def __init__(self, tensor_file, split="train"):
        data = torch.load(tensor_file)
        boards = data["boards"]
        evals = data["evals"]

        # Split logic: 70/10/20
        total = len(boards)
        if split == "train":
            self.boards = boards[:int(0.7 * total)]
            self.evals = evals[:int(0.7 * total)]
        elif split == "val":
            self.boards = boards[int(0.7 * total):int(0.8 * total)]
            self.evals = evals[int(0.7 * total):int(0.8 * total)]
        elif split == "test":
            self.boards = boards[int(0.8 * total):]
            self.evals = evals[int(0.8 * total):]
        else:
            raise ValueError(f"Unknown split: {split}")

    def __len__(self):
        return len(self.boards)

    def __getitem__(self, idx):
        return self.boards[idx], self.evals[idx]

# Model

In [93]:
class EvalNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(18, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(128 * 8 * 8, 512),
            nn.ReLU(),
            nn.Linear(512, 1)
        )

    def forward(self, x):
        return self.net(x)


model = EvalNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
loss_fn = torch.nn.MSELoss()
summary(model, input_size=(18, 8, 8))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1             [-1, 64, 8, 8]          10,432
              ReLU-2             [-1, 64, 8, 8]               0
            Conv2d-3            [-1, 128, 8, 8]          73,856
              ReLU-4            [-1, 128, 8, 8]               0
           Flatten-5                 [-1, 8192]               0
            Linear-6                  [-1, 512]       4,194,816
              ReLU-7                  [-1, 512]               0
            Linear-8                    [-1, 1]             513
Total params: 4,279,617
Trainable params: 4,279,617
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.26
Params size (MB): 16.33
Estimated Total Size (MB): 16.59
----------------------------------------------------------------


# Loaders

In [94]:
batch_size = 32
valid_games = 0.41

train_loader = DataLoader(PGNEvalDataset(pt_file, split="train"), batch_size=batch_size)
val_loader = DataLoader(PGNEvalDataset(pt_file, split="val"), batch_size=batch_size)
test_loader = DataLoader(PGNEvalDataset(pt_file, split="test"), batch_size=batch_size)

(2870, 410, 820)

# Training Loop

In [95]:
train_losses, val_losses = [], []
best_val_loss = float("inf")
plt.ion()

epochs = 10

for epoch in range(epochs):
    model.train()
    running_train = 0.0
    pbar = tqdm(
        train_loader, desc=f"[Epoch {epoch+1}] Training", total=len(train_loader), leave=False)

    for x, y in pbar:
        x, y = x.to(device), y.to(device)
        pred = model(x).squeeze()
        loss = loss_fn(pred, y.squeeze())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_train += loss.item()

        pbar.set_postfix({"batch_loss": f"{loss.item():.4f}"})

    avg_train = running_train / len(train_loader)
    train_losses.append(avg_train)

    # --- Validation ---
    model.eval()
    running_val = 0.0
    val_pbar = tqdm(
        val_loader, desc=f"[Epoch {epoch+1}] Validation", total=len(val_loader), leave=False)
    with torch.no_grad():
        num_val_batches = 0
        for x, y in val_pbar:
            x, y = x.to(device), y.to(device)
            pred = model(x).squeeze()
            loss = loss_fn(pred, y.squeeze())
            running_val += loss.item()
            num_val_batches += 1

            val_pbar.set_postfix({"val_loss": f"{loss.item():.4f}"})

    avg_val = running_val / len(val_loader)
    val_losses.append(avg_val)

    # --- Logging ---
    print(
        f"[Epoch {epoch+1}] ✅ Train Loss: {avg_train:.4f} | Val Loss: {avg_val:.4f}")

    # --- Checkpointing ---
    timestamp = time.strftime("%Y%m%d-%H%M%S")
    torch.save(model.state_dict(), os.path.join(model_dir, f"{timestamp}.pt"))
    if avg_val < best_val_loss:
        best_val_loss = avg_val
        torch.save(model.state_dict(), os.path.join(model_dir, "best.pt"))

    # --- Plotting ---
    plt.clf()
    plt.figure(figsize=(10, 6), dpi=150)
    plt.plot(train_losses, label="Train Loss")
    plt.plot(val_losses, label="Val Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Training & Validation Loss")
    plt.legend()
    plt.grid()
    plt.savefig(os.path.join(plot_dir, "loss_curve.png"))
    plt.close()

plt.ioff()
plt.show()

[Epoch 1] Training:   0%|          | 0/2870 [00:00<?, ?it/s]

[Epoch 1] Validation:   0%|          | 0/410 [00:00<?, ?it/s]

[Epoch 1] ✅ Train Loss: 16.4871 | Val Loss: 18.3479


[Epoch 2] Training:   0%|          | 0/2870 [00:00<?, ?it/s]

[Epoch 2] Validation:   0%|          | 0/410 [00:00<?, ?it/s]

[Epoch 2] ✅ Train Loss: 16.0373 | Val Loss: 17.1922


[Epoch 3] Training:   0%|          | 0/2870 [00:00<?, ?it/s]

[Epoch 3] Validation:   0%|          | 0/410 [00:00<?, ?it/s]

KeyboardInterrupt: 

<Figure size 640x480 with 0 Axes>

# Evaluation

In [ ]:
model.load_state_dict(torch.load(os.path.join(model_dir, "best.pt")))
model.eval()
test_loss = 0.0

print("\n🔍 Evaluating on test set...")

with torch.no_grad():
    pbar = tqdm(test_loader, desc="Testing", total=len(test_loader), leave=False)
    for x, y in pbar:
        x, y = x.to(device), y.to(device)
        pred = model(x).squeeze()
        loss = loss_fn(pred, y.squeeze())
        test_loss += loss.item()
        pbar.set_postfix({"batch_loss": f"{loss.item():.4f}"})

avg_test_loss = test_loss / len(test_loader)
print(f"\n🧪 Test Loss: {avg_test_loss:.4f}")